In [2]:
import analytics.api as ga
import analytics.sheets_api as sheets
import analytics.sheets_elements as elements
import analytics.entities as e
import pandas as pd
import gspread
from constants import *

In [3]:
%env ANALYTICS_REPORTING_CLIENT_SECRET_PATH=../../../../do_not_commit_ga4_credentials.json

env: ANALYTICS_REPORTING_CLIENT_SECRET_PATH=../../../../do_not_commit_ga4_credentials.json


In [4]:
ga_authentication, drive_authentication, sheets_authentication = ga.authenticate(
    SECRET_NAME,
    ga.ga4_service_params,
    ga.drive_service_params,
    ga.sheets_service_params,
    port=OAUTH_PORT
)

date_string = f"{START_DATE_CURRENT} - {END_DATE_CURRENT}"

default_params = {
    "service_system": ga_authentication,
    "start_date": START_DATE_CURRENT,
    "end_date": END_DATE_CURRENT,
}

lungmap_catalog_params = {
    **default_params,
    "property": LUNGMAP_ID,
}

lungmap_catalog_params_all_time = {
    **lungmap_catalog_params,
    "start_date": ANALYTICS_START,
    "end_date": END_DATE_CURRENT,
}

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=274560362763-p5netdrssq6r02lcfan6s157m6d65rqe.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8082%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly&state=u1KGZe2fXcNGFQvdEn0yUQvCbU4IHu&access_type=offline


In [5]:
df_monthly_pageviews = elements.get_page_views_over_time_df(lungmap_catalog_params_all_time, additional_data_path=HISTORIC_UA_DATA_PATH, additional_data_behavior=elements.ADDITIONAL_DATA_BEHAVIOR.ADD)
df_pageviews = elements.get_page_views_change(lungmap_catalog_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)
df_outbound = elements.get_outbound_links_change(lungmap_catalog_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)

In [6]:
dict_spreadsheet = {
    "Monthly Traffic Summary": df_monthly_pageviews,
    "Pageviews": df_pageviews,
    "Outbound Links": df_outbound,
}
sheet = sheets.create_sheet_in_folder(
        drive_authentication,
        SHEET_NAME,
        PARENT_FOLDER_NAME,
        override_behavior=sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE
    )
sheets.fill_spreadsheet_with_df_dict(
    sheet,
    dict_spreadsheet,
    sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE,
    column_formatting_options={
        "Monthly Traffic Summary": {
            e.DIMENSION_YEAR_MONTH["alias"]: sheets.COLUMN_FORMAT_OPTIONS.YEAR_MONTH_DATE,
            e.METRIC_ACTIVE_USERS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            e.METRIC_PAGE_VIEWS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Outbound Links": {
            e.SYNTHETIC_METRIC_CLICKS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            e.METRIC_TOTAL_USERS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Pageviews": {
            e.METRIC_PAGE_VIEWS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            e.METRIC_TOTAL_USERS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },

    },
    sheet_formatting_options={
        "Monthly Traffic Summary": {
            "extra_columns": 1,
            "extra_columns_width": 2000
        }
    }
)
monthly_traffic_worksheet = sheet.worksheet("Monthly Traffic Summary")
date_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 1), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 2)
)
users_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 2), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 3)
)
pageviews_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 3), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 4)
)
sheets.add_chart_to_sheet(
    sheets_authentication,
    sheet,
    sheet.worksheet("Monthly Traffic Summary"),
    sheets.CHART_TYPES.LINE,
    date_range,
    [users_range, pageviews_range],
    chart_position=gspread.cell.Cell(1, 6),
    chart_position_offset_x=75,
    chart_position_offset_y=75,
    title="Pageviews and Users Over Time"
)

{'spreadsheetId': '1LC6lrC3ZhGuJFtPFIgvNWcFeoGyQnQtIkFAckEliVG0',
 'replies': [{'addChart': {'chart': {'chartId': 678937881,
     'spec': {'title': 'Pageviews and Users Over Time',
      'basicChart': {'chartType': 'LINE',
       'axis': [{'position': 'BOTTOM_AXIS', 'viewWindowOptions': {}},
        {'position': 'LEFT_AXIS', 'viewWindowOptions': {}}],
       'domains': [{'domain': {'sourceRange': {'sources': [{'sheetId': 286312932,
             'startRowIndex': 0,
             'endRowIndex': 41,
             'startColumnIndex': 0,
             'endColumnIndex': 1}]}}}],
       'series': [{'series': {'sourceRange': {'sources': [{'sheetId': 286312932,
             'startRowIndex': 0,
             'endRowIndex': 41,
             'startColumnIndex': 1,
             'endColumnIndex': 2}]}},
         'targetAxis': 'LEFT_AXIS'},
        {'series': {'sourceRange': {'sources': [{'sheetId': 286312932,
             'startRowIndex': 0,
             'endRowIndex': 41,
             'startColumnIndex